# Comparing Different Approaches to Breast Cancer Ultrasound Diagnosis using Deep Learning


## Team Members
- Samantha Guillemette - ID: 26609198
- Saleha Tariq - ID: 40006997

# Librairies

# Data preparation & pre-processing

# Model Class Definitions

# Model 1

# Model 2